In [1]:
import pandas as pd
from datetime import date
import numpy as np
from dateutil.parser import parse 

购买时差处理

In [12]:
order_data=pd.read_excel('唯衣POC.xlsx',sheet_name=0,header=0)#订单表
item_info=pd.read_excel('唯衣POC.xlsx',sheet_name=2,header=1)#商品信息表
item_id=pd.read_excel('唯衣POC.xlsx',sheet_name=3,header=0)#条码id表
order_data=order_data.drop(0)
item_id=item_id.drop(0)

In [13]:
#得到订单商品表
user_item=pd.merge(order_data,item_id,left_on=['订单所含商品条码ID'],right_on=['条码id'],how='left')

In [14]:
user_item=user_item.drop(['订单总金额','订单所含商品单价','条码'],axis=1)

In [15]:
item_info_date=item_info[['ID','款号','年份','季节']]

In [16]:
date_map={'春装':'01','夏装':'04','秋装':'07','冬装':'10','全年':'02',np.nan:np.nan}#季节月份对应关系

In [17]:
def date_process(item_info):    
    if item_info['年份']!=np.NaN:                  
            y=item_info['年份'][:4]
            m=date_map[item_info['季节']]            
            item_info['衣服上市时间']=pd.Period(parse(y+'-'+m),freq='M') 
    return item_info
item_info_date=item_info_date.apply(date_process,axis=1)#计算衣服上市时间

In [18]:
#得到订单商品信息表，包含了衣服上市时间
order_item_info=pd.merge(user_item,item_info_date,left_on=['款号'],right_on=['款号'],how='left')

In [57]:
order_item_info['aaa']= pd.to_datetime(order_item_info['订单时间'],format='%Y%m%d') .resample('M') 
order_item_info.loc[1,'aaa']

ValueError: cannot specify both format and unit

In [25]:
#计算时差
order_item_info['购买时差']=0
def compute_date(df):
    if type(df['年份'])==str:
        x=df['订单时间']
        date=pd.Period(parse(str(x)),freq='M')
        df['购买时差']=date-df['衣服上市时间']
    return df
order_item_info=order_item_info.apply(compute_date,axis=1)

In [26]:
len(order_item_info[order_item_info['购买时差']>-0])

376126

In [68]:
 order_item_info.to_csv("购买时差表.csv")

以下为面料处理

In [34]:
item_info_fabric=item_info[['ID','面料成分']]

面 料：聚酯纤维100%&罗 纹：粘纤75.7%;锦纶22.2%;氨纶2.1%

[\u4e00-\u9fa5]+[0-9.]+[%]#匹配正则表达式，得到'粘纤75.7%'的列表

[\u4e00-\u9fa5]{1,}#中文匹配，得到里面的中文

In [35]:
import re
re_all = re.compile(r'[\u4e00-\u9fa5]+[0-9.]+[%]')#提取‘粘纤75.7%’
re_name=re.compile(r'[\u4e00-\u9fa5]{1,}')#提取成分名称
re_num=re.compile(r'[0-9.]+')#提取百分比
def fabric_process(item_info):
    s=item_info['面料成分']
    if type(s)==str:        
        list_all=re_all.findall(s)    
        for x in list_all:        
            name = re_name.findall(x) 
            num = re_num.findall(x) 
            item_info[name[0]]=num[0]
    return item_info
item_info_fabric=item_info_fabric.apply(fabric_process,axis=1)

In [30]:
item_info_fabric.drop(['面料成分'],axis=1,inplace=True)

In [31]:
item_info_fabric=item_info_fabric.fillna(0)

In [93]:
item_info_fabric.to_csv('面料成分解析.csv')